# 案例: 带扰动的分层控制系统

本案例演示了在面临外部扰动（突发的降雨事件）时，分层控制系统的韧性。场景与基础的分层控制案例相同，但增加了：

- 一个`RainfallAgent`，它会在仿真中途模拟一次突然的大流量入库。
- 一个能感知消息并订阅扰动主题的`Reservoir`模型。

这个例子展示了解耦的智能体如何对意外事件作出反应以维持系统稳定。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# 导入平台组件
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher
from swp.simulation_identification.disturbances.rainfall_agent import RainfallAgent

CONFIG_PATH = 'examples/hierarchical_control_with_disturbance.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义系统搭建和运行函数

In [ ]:
def setup_and_run_simulation(cfg):
    # 1. 仿真和通信设置
    harness = SimulationHarness(config=cfg['simulation'])
    message_bus = harness.message_bus
    sim_cfg = cfg['simulation']
    topics = cfg['messaging_topics']
    comp_cfgs = cfg['components']
    agent_cfgs = cfg['agents']
    
    # 2. 创建核心物理组件
    res_cfg = comp_cfgs['reservoir']
    # 让水库能感知扰动
    reservoir = Reservoir("reservoir_1", res_cfg['initial_state'], res_cfg['params'], message_bus=message_bus, disturbance_topics=[topics['rainfall_disturbance']])
    gate_cfg = comp_cfgs['gate']
    gate = Gate("gate_1", gate_cfg['initial_state'], gate_cfg['params'], message_bus, topics['gate_action'])

    # 3. 创建核心控制智能体 (与基础版分层控制相同)
    reservoir_twin = DigitalTwinAgent("twin_agent_reservoir_1", reservoir, message_bus, topics['reservoir_state'])
    gate_twin = DigitalTwinAgent("twin_agent_gate_1", gate, message_bus, topics['gate_state'])
    pid_cfg = agent_cfgs['pid_controller']
    pid_controller = PIDController(setpoint=pid_cfg['setpoint'], Kp=pid_cfg['Kp'], Ki=pid_cfg['Ki'], Kd=pid_cfg['Kd'], min_output=0.0, max_output=gate_cfg['params']['max_opening'])
    control_agent = LocalControlAgent("control_agent_gate_1", pid_controller, message_bus, topics['reservoir_state'], 'water_level', topics['gate_action'], sim_cfg['dt'], topics['gate_command'], topics['gate_state'])
    disp_cfg = agent_cfgs['dispatcher']
    dispatcher = CentralDispatcher("central_dispatcher_1", message_bus, {'reservoir_level': topics['reservoir_state']}, {'gate1_command': topics['gate_command']}, disp_cfg['rules'])

    # 4. 创建扰动智能体
    rain_cfg = agent_cfgs['rainfall_disturbance']
    rainfall_config = {**rain_cfg, 'topic': topics['rainfall_disturbance']}
    rainfall_agent = RainfallAgent("rainfall_agent_1", message_bus, rainfall_config)

    # 5. 添加所有组件和智能体到仿真平台
    for component in [reservoir, gate]: harness.add_component(component)
    for agent in [reservoir_twin, gate_twin, control_agent, dispatcher, rainfall_agent]: harness.add_agent(agent)

    # 6. 定义拓扑并运行
    harness.add_connection("reservoir_1", "gate_1")
    harness.build()
    print("\n--- 系统搭建完毕，开始运行仿真 ---")
    harness.run_mas_simulation()
    print("--- 仿真结束 ---")
    return harness.history

history = setup_and_run_simulation(config)

### 3. 可视化结果

下图清晰地展示了在第300秒扰动开始后，系统是如何响应的。中央调度器检测到水位将超过洪水阈值，于是下调了PID控制器的目标水位，使得水闸开度增大，从而预泄洪，将水位控制在安全范围内。

In [ ]:
def plot_disturbance_results(simulation_history, cfg):
    if not simulation_history:
        print("没有历史数据可供绘制。")
        return
        
    # 数据转换
    flat_history = []
    for step in simulation_history:
        row = {'time': step['time']}
        for component_name, states in step.items():
            if component_name == 'time' or not isinstance(states, dict): continue
            for state_key, value in states.items():
                row[f"{component_name}.{state_key}"] = value
        flat_history.append(row)
    results_df = pd.DataFrame(flat_history).set_index('time')
    
    # 绘图
    fig, ax1 = plt.subplots(figsize=(15, 7))
    fig.suptitle('带扰动的分层控制仿真结果', fontsize=16)

    ax1.plot(results_df.index, results_df['reservoir_1.water_level'], 'b-', label='水库水位 (m)')
    ax1.set_xlabel('时间 (s)')
    ax1.set_ylabel('水位 (m)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.grid(True)
    ax1.axhline(y=cfg['agents']['dispatcher']['rules']['flood_threshold'], color='r', linestyle=':', label='洪水阈值')

    # 突出显示扰动区间
    rain_cfg = cfg['agents']['rainfall_disturbance']
    ax1.axvspan(rain_cfg['start_time'], rain_cfg['start_time'] + rain_cfg['duration'], facecolor='orange', alpha=0.3, label='降雨扰动期')

    ax2 = ax1.twinx()
    ax2.plot(results_df.index, results_df['gate_1.opening'], 'g-', label='水闸开度 (m)')
    ax2.set_ylabel('开度 (m)', color='g')
    ax2.tick_params(axis='y', labelcolor='g')

    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    plt.show()

plot_disturbance_results(history, config)